In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import EstimatorPreprocessor as ep
from sklearn.metrics import accuracy_score

In [8]:
data = ep.load_cleaned_submissions()

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Encode labels
from sklearn.preprocessing import LabelEncoder
lenc = LabelEncoder()
y = lenc.fit_transform(data["sentiment"])

# Vectorize text using tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(preprocessor=' '.join, lowercase=False, min_df=5) # min_df = Minimum occurance of words
X = tfidf.fit_transform(data["text"])

In [10]:
# Using Vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# for sentence in data["text"][indices][:30]:
#     vs = analyzer.polarity_scores(" ".join(sentence))
#     print("{:} {}".format(" ".join(sentence), str(vs)))
#     print("-"*40)


def vader_sentiment_labels(query_text, compound_score):
    sentiment_list = []
    for item in query_text:
        sentence = tfidf.inverse_transform(item)
        vs = analyzer.polarity_scores(" ".join(sentence[0]))
        if vs["compound"] > compound_score:
            sentiment_list.append(np.int(2)) # Positive Sentiment
        elif vs["compound"] < -compound_score:
            sentiment_list.append(np.int(0)) # Negative Sentiment
        else:
            sentiment_list.append(np.int(1)) # Neutral Sentiment
    return sentiment_list

In [16]:
compound_scores = [0.10] #[0.03, 0.05, 0.07, 0.10]

In [17]:
history = []
s = ""
for score in compound_scores:
    y_pred = vader_sentiment_labels(X, score)
    acc = accuracy_score(y, y_pred)
    s = "Score: " + str(s) + ";Accuracy:" + str(acc)
    history.append(s)

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred)

0.38837276656418485

In [15]:
y_pred = vader_sentiment_labels(X, 0.10)

KeyboardInterrupt: 

In [23]:
# Use best params of model for classification report
from sklearn.metrics import classification_report
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.30      0.52      0.38     24088
           1       0.64      0.38      0.48    121113
           2       0.14      0.28      0.19     34343

    accuracy                           0.38    179544
   macro avg       0.36      0.39      0.35    179544
weighted avg       0.50      0.38      0.41    179544

